In [5]:
import os
import datetime
from pymongo import MongoClient
import json
import traceback

# Configuración mejorada de la conexión a MongoDB con manejo de errores
try:
    MONGO_URI = "mongodburi"
    client = MongoClient(MONGO_URI, serverSelectionTimeoutMS=5000)
    db = client.get_database("Test_Nayeli")
    client.server_info()  # Verifica la conexión al servidor
except Exception as e:
    print(f"Error de conexión a MongoDB: {str(e)}")
    exit(1)

# Configuración de la estructura de carpetas con timestamp
backup_base = "./bk"
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
backup_folder = os.path.join(backup_base, f"backup_{timestamp}")

try:
    os.makedirs(backup_folder, exist_ok=True)
except OSError as e:
    print(f"Error creando directorio de respaldo: {str(e)}")
    exit(1)

# Función para respaldar una colección individual
def backup_collection(collection_name, backup_dir):
    try:
        collection = db.get_collection(collection_name)
        data = list(collection.find())
        
        if not data:
            print(f"Colección {collection_name} vacía, omitiendo...")
            return True
            
        backup_file = os.path.join(backup_dir, f"{collection_name}.json")
        
        with open(backup_file, 'w') as file:
            json.dump(data, file, default=str, indent=2)
            
        print(f"Respaldo de {collection_name} completado: {backup_file}")
        return True
        
    except Exception as e:
        print(f"Error respaldando {collection_name}: {str(e)}")
        traceback.print_exc()
        return False

# Proceso principal de respaldo
success_count = 0
failure_count = 0

try:
    collections = db.list_collection_names()
    print(f"Iniciando respaldo de {len(collections)} colecciones...")
    
    for collection_name in collections:
        if backup_collection(collection_name, backup_folder):
            success_count += 1
        else:
            failure_count += 1
            
    print(f"\nResumen del respaldo:")
    print(f"Colecciones respaldadas exitosamente: {success_count}")
    print(f"Colecciones con errores: {failure_count}")
    print(f"Ubicación del respaldo: {backup_folder}")
    
except Exception as e:
    print(f"Error general en el proceso de respaldo: {str(e)}")
    exit(1)


Iniciando respaldo de 5 colecciones...
Respaldo de resultados completado: ./bk/backup_20250209_123901/resultados.json
Respaldo de pruebas completado: ./bk/backup_20250209_123901/pruebas.json
Respaldo de users completado: ./bk/backup_20250209_123901/users.json
Respaldo de escuelas completado: ./bk/backup_20250209_123901/escuelas.json
Respaldo de carreras completado: ./bk/backup_20250209_123901/carreras.json

Resumen del respaldo:
Colecciones respaldadas exitosamente: 5
Colecciones con errores: 0
Ubicación del respaldo: ./bk/backup_20250209_123901
